# Breaching privacy

This notebook does the same job as the cmd-line tool `simulate_breach.py`, but also directly visualizes the user data and reconstruction

In [1]:
import torch
import hydra
from omegaconf import OmegaConf
%load_ext autoreload
%autoreload 2

import breaching
import logging, sys
logging.basicConfig(level=logging.INFO, handlers=[logging.StreamHandler(sys.stdout)], format='%(message)s')
logger = logging.getLogger()

### Initialize cfg object and system setup:

This will print out all configuration options. 
There are a lot of possible configurations, but there is usually no need to worry about most of these. Below, a few options are printed.

Choose `case/data=` `shakespeare`, `wikitext`over `stackoverflow` here:

In [2]:
with hydra.initialize(config_path="config"):
    cfg = hydra.compose(config_name='cfg', overrides=["case/data=wikitext", "case/server=malicious-transformer",
                                                      "case.model=transformer3",
                                                      "attack=decepticon"])
    print(f'Investigating use case {cfg.case.name} with server type {cfg.case.server.name}.')
          
device = torch.device('cpu')
torch.backends.cudnn.benchmark = cfg.case.impl.benchmark
setup = dict(device=device, dtype=torch.float)
setup

Investigating use case single_imagenet with server type malicious_transformer_parameters.


{'device': device(type='cpu'), 'dtype': torch.float32}

### Modify config options here

You can use `.attribute` access to modify any of these configurations:

In [3]:
cfg.case.user.num_data_points = 8 # How many sentences?
cfg.case.user.user_idx = 1 # From which user?
cfg.case.data.shape = [512] # This is the sequence length

cfg.case.server.has_external_data = True
cfg.case.data.tokenizer = "gpt2"

### Instantiate all parties

In [4]:
user, server, model, loss_fn = breaching.cases.construct_case(cfg.case, setup)
attacker = breaching.attacks.prepare_attack(server.model, server.loss, cfg.attack, setup)
breaching.utils.overview(server, user, attacker)

Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Model architecture transformer3 loaded with 10,800,433 parameters and 0 buffers.
Overall this is a data ratio of    2637:1 for target shape [8, 512] given that num_queries=1.
User (of type UserSingleStep) with settings:
    Number of data points: 8

    Threat model:
    User provides labels: False
    User provides buffers: False
    User provides number of data points: True

    Data:
    Dataset: wikitext
    user: 1
    
        
Server (of type MaliciousTransformerServer) with settings:
    Threat model: Malicious (Parameters)
    Number of planned queries: 1
    Has external/public data: True

    Model:
        model specification: transformer3
        model state: default
        

    Secrets: {}
    


### Simulate an attacked FL protocol

True user data is returned only for analysis

In [5]:
server_payload = server.distribute_payload()
shared_data, true_user_data = user.compute_local_updates(server_payload)

Found attention of shape torch.Size([288, 96]).
Computing feature distribution before the probe layer Linear(in_features=96, out_features=1536, bias=True) from external data.
Feature mean is -0.12308123707771301, feature std is 1.0215680599212646.
Computing user update in model mode: eval.


In [15]:
#user.print(true_user_data)

# Reconstruct user data

In [25]:
attacker.cfg.embedding_token_weight = 0.0
attacker.cfg.sentence_algorithm = "dynamic-threshold"

In [26]:
reconstructed_user_data, stats = attacker.reconstruct([server_payload], [shared_data], 
                                                      server.secrets, dryrun=cfg.dryrun)
#user.print(reconstructed_user_data)
metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, cfg_case=cfg.case, setup=setup)

Recovered tokens tensor([[   11,    12,    13,  ...,  3365,  3378,  3386],
        [ 3388,  3389,  3392,  ..., 19683, 19685, 19954],
        [  262,   262,   262,  ..., 49889, 50203, 50210],
        ...,
        [   11,    11,    11,  ..., 49658, 49658, 49658],
        [   11,    11,    11,  ..., 49658, 49658, 49658],
        [   11,    12,    13,  ..., 48405, 49658, 50210]]) through strategy decoder-bias.
Recovered 2694 embeddings with positional data from imprinted layer.
Assigned [310, 88, 512, 349, 348, 326, 313, 448] breached embeddings to each sentence.
METRICS: | Accuracy: 0.3916 | S-BLEU: 0.14 | FMSE: 2.4272e-01 | 
 G-BLEU: 0.19 | ROUGE1: 0.57| ROUGE2: 0.15 | ROUGE-L: 0.37| Token Acc: 99.37% | Label Acc: 99.37%


In [7]:
reconstructed_user_data, stats = attacker.reconstruct([server_payload], [shared_data], 
                                                      server.secrets, dryrun=cfg.dryrun)
#user.print(reconstructed_user_data)
metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, cfg_case=cfg.case, setup=setup)

Recovered tokens tensor([[   11,    12,    13,  ...,  3365,  3378,  3386],
        [ 3388,  3389,  3392,  ..., 19683, 19685, 19954],
        [  262,   262,   262,  ..., 49889, 50203, 50210],
        ...,
        [   11,    11,    11,  ..., 49658, 49658, 49658],
        [   11,    11,    11,  ..., 49658, 49658, 49658],
        [   11,    12,    13,  ..., 48405, 49658, 50210]]) through strategy decoder-bias.
Recovered 2694 embeddings with positional data from imprinted layer.
Assigned [310, 88, 512, 349, 348, 326, 313, 448] breached embeddings to each sentence.
 arms, he sent " formal demand for rank of lieutenant arsenal to Captain ordotten Arsenal, 1893 This the is prompted by the feeling that perv obsolete the citizens of this State that in the, emergency the of dutiesA of war in < Arsenal North be Rock themission of the State authorities ) in various t their exclusively Records This the, and not  of me History has documents became an indicating or it acres69 duty at as� Summary of th

METRICS: | Accuracy: 0.3916 | S-BLEU: 0.14 | FMSE: 2.4272e-01 | 
 G-BLEU: 0.19 | ROUGE1: 0.57| ROUGE2: 0.15 | ROUGE-L: 0.37| Token Acc: 99.37% | Label Acc: 99.37%


In [8]:
reconstructed_user_data, stats = attacker.reconstruct_token_first([server_payload], [shared_data], 
                                                      server.secrets, dryrun=cfg.dryrun)
#user.print(reconstructed_user_data)
metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, cfg_case=cfg.case, setup=setup)

Recovered tokens tensor([[   11,    12,    13,  ...,  3365,  3378,  3386],
        [ 3388,  3389,  3392,  ..., 19683, 19685, 19954],
        [  262,   262,   262,  ..., 49889, 50203, 50210],
        ...,
        [   11,    11,    11,  ..., 49658, 49658, 49658],
        [   11,    11,    11,  ..., 49658, 49658, 49658],
        [   11,    12,    13,  ..., 48405, 49658, 50210]]) through strategy decoder-bias.
Recovered 2694 embeddings with positional data from imprinted layer.
Assigned [310, 88, 512, 349, 348, 326, 313, 448] breached embeddings to each sentence.
 arms, he sent " formal demand for rank of lieutenant arsenal to Captain ordotten 000, 1893 This the is prompted by the feeling that perv held the citizens of this State that in the, emergency the arms dutiesA of war in < Arsenal North be Rock themission of the State authorities ) in order garrison their exclusively Records This the, although not Mas of me, has assumed such an indicating or it acres69 duty be as� arsenal of this t

METRICS: | Accuracy: 0.4573 | S-BLEU: 0.20 | FMSE: 2.1483e-01 | 
 G-BLEU: 0.23 | ROUGE1: 0.61| ROUGE2: 0.21 | ROUGE-L: 0.43| Token Acc: 85.13% | Label Acc: 85.13%


In [12]:
attacker.cfg.embedding_token_weight = 0.25
reconstructed_user_data, stats = attacker.reconstruct([server_payload], [shared_data], 
                                                      server.secrets, dryrun=cfg.dryrun)
#user.print(reconstructed_user_data)
metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, cfg_case=cfg.case, setup=setup)

Recovered tokens tensor([[   11,    12,    13,  ...,  3365,  3378,  3386],
        [ 3388,  3389,  3392,  ..., 19683, 19685, 19954],
        [  262,   262,   262,  ..., 49889, 50203, 50210],
        ...,
        [   11,    11,    11,  ..., 49658, 49658, 49658],
        [   11,    11,    11,  ..., 49658, 49658, 49658],
        [   11,    12,    13,  ..., 48405, 49658, 50210]]) through strategy decoder-bias.
Recovered 2694 embeddings with positional data from imprinted layer.
Assigned [310, 88, 512, 349, 348, 326, 313, 448] breached embeddings to each sentence.
Replaced 13 tokens with avg. corr 0.19007538259029388 with new tokens with avg corr 0.8640716075897217
METRICS: | Accuracy: 0.3938 | S-BLEU: 0.15 | FMSE: 2.4193e-01 | 
 G-BLEU: 0.19 | ROUGE1: 0.57| ROUGE2: 0.15 | ROUGE-L: 0.37| Token Acc: 99.19% | Label Acc: 99.19%


In [30]:
# Decorr tests
import numpy as np

In [564]:
a = torch.randn(64) * 3- 0.5
b = torch.randn(64) * 0.5 + 0.3
c = torch.randn(64) - 1

base = torch.stack([a, b, c])
base = base - base.mean(dim=-1, keepdim=True)

In [565]:
np.corrcoef(a+b, b)[0, 1], np.corrcoef(a, b)[0, 1], np.corrcoef(a, c)[0, 1]

(0.4607779430939982, 0.3180582941992394, 0.0975540912719207)

In [566]:
true = [1, 1, 0, 0, 1, 2, 2, 2, 2, 0, 1]

In [567]:
A = torch.stack([a+b, b, a, a*2, 3*b, c, c, 3*c, -c, c-a, a+3*b])[None]
A_1 = A - A.mean(dim=-1, keepdim=True)

U, S, V = torch.pca_lowrank(A, q=3, center=False, niter=20)

In [568]:
A.shape, U.shape, S, V.shape, a_c.shape 

(torch.Size([1, 11, 64]),
 torch.Size([1, 11, 3]),
 tensor([[73.6484, 33.3310, 14.7601]]),
 torch.Size([1, 64, 3]),
 torch.Size([1, 5, 2]))

In [569]:
np.round(100 * np.abs(np.corrcoef(V[0, :, :].T, base)[3:, :3]))

array([[100.,  23.,   5.],
       [ 40.,  65.,  69.],
       [  7.,  90.,  35.]])

In [570]:
U.shape, S.shape

(torch.Size([1, 11, 3]), torch.Size([1, 3]))

In [571]:
U.abs().argmax(dim=-1)

tensor([[0, 2, 0, 0, 2, 1, 1, 1, 1, 0, 2]])

In [558]:
U.div(S[None]).abs().argmax(dim=-1)

tensor([[2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 0]])

In [559]:
A = torch.stack([a, b, c, a*2, 3*b, a+b, c])[None]
A_1 = A - A.mean(dim=-1, keepdim=True)
U, S, V = torch.linalg.svd(A_1,full_matrices=False)
U.shape, S.shape, V.shape, S

(torch.Size([1, 7, 7]),
 torch.Size([1, 7]),
 torch.Size([1, 7, 64]),
 tensor([[2.8729e+01, 1.7385e+01, 9.9125e+00, 2.5548e-06, 1.5805e-06, 5.9307e-07,
          4.3564e-07]]))

In [560]:
uncorr = U[:, :3, :].matmul(A_1)

In [561]:
np.round(100 * np.abs(np.corrcoef(uncorr[0, :, :], torch.stack([a, b, c]))[3:, :3]))

array([[ 95.,  33.,  36.],
       [  2., 100.,  12.],
       [  5.,   6.,  96.]])

In [562]:
U

tensor([[[-1.6182e-01, -3.7643e-01, -3.6735e-02,  8.5353e-01, -1.3752e-01,
           2.4842e-01,  1.4706e-01],
         [-2.6250e-01,  1.5228e-01,  1.4654e-02,  1.1245e-02,  8.4249e-01,
           4.3505e-01,  9.2263e-02],
         [ 4.4381e-04,  6.8491e-02, -7.0378e-01,  4.5795e-03, -8.9888e-02,
           3.0762e-01, -6.3029e-01],
         [-3.2363e-01, -7.5285e-01, -7.3470e-02, -4.9824e-01, -8.8320e-02,
           2.2903e-01,  1.2076e-01],
         [-7.8749e-01,  4.5684e-01,  4.3962e-02, -5.1395e-02, -3.8555e-01,
           9.0477e-02,  9.8769e-02],
         [-4.2431e-01, -2.2415e-01, -2.2081e-02,  1.4294e-01,  3.1416e-01,
          -7.0648e-01, -3.8857e-01],
         [ 4.4386e-04,  6.8491e-02, -7.0378e-01, -4.5796e-03,  8.9887e-02,
          -3.0762e-01,  6.3029e-01]]])

# Now finally decorrelate two things

In [671]:
a = torch.randn(32) * 3 - 0.5
b = torch.randn(32) * 0.5 + 0.3

In [672]:
np.corrcoef(a, b)[0, 1], np.corrcoef(a+b, a)[0, 1], np.corrcoef(a+b, b)[0, 1]

(0.012967353968462872, 0.9894317670582278, 0.15781735610279837)

In [673]:
A = torch.stack([a+b, b])[None]
U, S, V = torch.pca_lowrank(A - A.mean(dim=-1, keepdim=True), q=1, center=False, niter=2)

In [674]:
np.corrcoef(V[0, :, 0], a)[0, 1], np.corrcoef(V[0, :, 0], b)[0, 1], np.corrcoef(V[0, :, 0], a+b)[0, 1]

(0.988837129428431, 0.16181051431948437, 0.9999918183802636)

tensor([[[0.9997],
         [0.0235]]])

In [676]:
ab = a + b
std, mean = torch.std_mean(ab)
std2, mean2 = torch.std_mean(b)

In [677]:
decorr = (ab - mean) / std - np.corrcoef(a+b, b)[0, 1] * (b - mean2) / std2

In [678]:
# decorr = ab - np.corrcoef(a+b, b)[0, 1] * b

In [679]:
np.corrcoef(decorr, a)[0, 1], np.corrcoef(decorr, b)[0, 1]

(0.9999159201448755, -1.4340345536158765e-08)

In [680]:
np.corrcoef(a+b, b)[0, 1]

0.15781735610279837

In [682]:
((ab - mean) / std * (b - mean2) / std2).sum() / ((ab - mean) / std).norm() / ((b - mean2) / std2).norm()

tensor(0.1578)